### Set Up Packages and Global Settings

In [1]:
import pytz
import random
import time

from datetime import datetime

import fastf1 as ff1
import pandas as pd

pd.set_option("display.max_columns", None)

### Explore Schedule

In [ ]:
schedule = ff1.get_event_schedule(1994)
schedule.RoundNumber.to_list()

### Explore Session Object

In [ ]:
session_quali = ff1.get_session(2020, 1, "Q")

session_quali.load()
session_quali.event

In [ ]:
session_quali = ff1.get_session(2020, 1, 'R')

session_quali.load()
session_quali.results

### Explore Laps

In [ ]:
lap_data = session_number.laps
print(lap_data)

In [ ]:
lap_data.pick_team('McLaren').pick_fastest()

In [ ]:
lap_data.pick_drivers(['PIA', 'NOR'])

### Compare DriverId field to names

In [ ]:
df_all = pd.DataFrame()

list_years = list(range(1994, 1995 + 1))

max_retries = 5

for year in list_years:
    print(f"Pulling data for {year}")

    schedule = ff1.get_event_schedule(year)
    rounds_all = schedule.RoundNumber.to_list()
    rounds_races = [round for round in rounds_all if round > 0]

    for round in rounds_races:
        for attempt in range(max_retries):
            try:
                now_utc = datetime.now(pytz.timezone("UTC"))
                now_cst = now_utc.astimezone(pytz.timezone("America/Chicago"))

                print(f"Pulling data for round {round} at {now_cst}...")

                session_quali = ff1.get_session(year, round, "Q")
                session_quali.load()
                df_quali = session_quali.results[["DriverId", "LastName", "FirstName"]]

                df_all = df_all.append(df_quali, ignore_index=True)
                time.sleep(5)

                session_race = ff1.get_session(year, round, "R")
                session_race.load()
                df_race = session_race.results[["DriverId", "LastName", "FirstName"]]

                df_all = df_all.append(df_race, ignore_index=True)
                time.sleep(5)

                df_all = df_all.drop_duplicates().reset_index(drop=True)

                # If no exception was raised, break the loop
                break
            except Exception as e:
                wait_time = (2**attempt) + random.random()
                print(
                    f"Could not load data for round {round} due to {e}. Retrying in {wait_time} seconds."
                )
                time.sleep(wait_time)
                continue

df_unique = df_all.sort_values(by="DriverId")

### Develop Approach for Quali Results Object

In [30]:
session_year = 1994
session_round = 1

session_quali = ff1.get_session(session_year, session_round, "Q")

session_quali.load()
df_raw = session_quali.results[
    ["Q1", "Q2", "Q3", "DriverId", "LastName", "FirstName", "TeamName", "Position"]
]

df_processed = df_raw.melt(
    id_vars=["DriverId", "LastName", "FirstName", "TeamName", "Position"],
    value_vars=["Q1", "Q2", "Q3"],
    var_name="session",
    value_name="time",
)

df_processed.columns = df_processed.columns.str.lower()

df_processed["year"] = session_year
df_processed["round"] = session_round

df_processed["originalposition"] = df_processed["position"]
df_processed["position"] = df_processed.groupby("session")["time"].rank(
    method="min", ascending=True
)
df_processed["position"] = df_processed["position"].fillna(
    df_processed["originalposition"]
)

df_processed = df_processed.drop(columns=["originalposition"])

df_processed.rename(
    columns={
        "driverid": "id_driver",
        "lastname": "name_driver_last",
        "firstname": "name_driver_first",
        "teamname": "name_team",
    },
    inplace=True,
)
df_quali_final = df_processed[
    [
        "year",
        "round",
        "id_driver",
        "name_driver_last",
        "name_driver_first",
        "name_team",
        "session",
        "position",
        "time",
    ]
]

df_quali_final

core           INFO 	Loading data for Brazilian Grand Prix - Qualifying [v3.1.6]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
logger      WARNING 	Failed to load session info data!
core        WARNING 	Cannot load laps, telemetry, weather, and message data because the relevant API is not supported for this session.
core           INFO 	Finished loading data for 26 drivers: ['2', '5', '27', '0', '30', '10', '29', '7', '6', '3', '9', '4', '20', '14', '23', '15', '28', '8', '26', '25', '12', '24', '19', '11', '34', '31']


,year,round,id_driver,name_driver_last,name_driver_first,name_team,session,position,time
0,1994,1,senna,Senna,Ayrton,Williams,Q1,1.0,0 days 00:01:15.962000
1,1994,1,michael_schumacher,Schumacher,Michael,Benetton,Q1,2.0,0 days 00:01:16.290000
2,1994,1,alesi,Alesi,Jean,Ferrari,Q1,3.0,0 days 00:01:17.385000
3,1994,1,damon_hill,Hill,Damon,Williams,Q1,4.0,0 days 00:01:17.554000
4,1994,1,frentzen,Frentzen,Heinz-Harald,Sauber,Q1,5.0,0 days 00:01:17.806000
...,...,...,...,...,...,...,...,...,...
73,1994,1,alboreto,Alboreto,Michele,Minardi,Q3,22.0,NaT
74,1994,1,beretta,Beretta,Olivier,Larrousse,Q3,23.0,NaT
75,1994,1,lamy,Lamy,Pedro,Team Lotus,Q3,24.0,NaT
76,1994,1,gachot,Gachot,Bertrand,Pacific,Q3,25.0,NaT


### Develop Approach for Race Results Object

In [31]:
session_year = 1994
session_round = 1

session_race = ff1.get_session(session_year, session_round, "R")

session_race.load()
df_raw = session_race.results[
    [
        "DriverId",
        "LastName",
        "FirstName",
        "TeamName",
        "ClassifiedPosition",
        "Time",
    ]
]

df_processed = df_raw.copy().reset_index(drop=True)

df_processed["year"] = session_year
df_processed["round"] = int(session_round)
df_processed["session"] = "Race"

df_processed.columns = df_processed.columns.str.lower()
df_processed.rename(
    columns={
        "classifiedposition": "position",
        "driverid": "id_driver",
        "lastname": "name_driver_last",
        "firstname": "name_driver_first",
        "teamname": "name_team",
    },
    inplace=True,
)
df_processed.loc[1:, "time"] = (
    df_processed.loc[1:, "time"] + df_processed.loc[0, "time"]
)
df_race_final = df_processed[
    [
        "year",
        "round",
        "session",
        "id_driver",
        "name_driver_last",
        "name_driver_first",
        "name_team",
        "position",
        "time",
    ]
]

df_race_final

core           INFO 	Loading data for Brazilian Grand Prix - Race [v3.1.6]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
logger      WARNING 	Failed to load session info data!
core        WARNING 	Cannot load laps, telemetry, weather, and message data because the relevant API is not supported for this session.
core           INFO 	Finished loading data for 28 drivers: ['5', '0', '27', '14', '3', '29', '12', '23', '20', '11', '26', '31', '2', '8', '15', '6', '25', '4', '9', '30', '7', '24', '10', '28', '19', '34', '32', '33']


,year,round,session,id_driver,name_driver_last,name_driver_first,name_team,position,time
0,1994,1,Race,michael_schumacher,Schumacher,Michael,Benetton,1,0 days 01:35:39.200000
1,1994,1,Race,damon_hill,Hill,Damon,Williams,2,NaT
2,1994,1,Race,alesi,Alesi,Jean,Ferrari,3,NaT
3,1994,1,Race,barrichello,Barrichello,Rubens,Jordan,4,NaT
4,1994,1,Race,katayama,Katayama,Ukyo,Tyrrell,5,NaT
5,1994,1,Race,wendlinger,Wendlinger,Karl,Sauber,6,NaT
6,1994,1,Race,herbert,Herbert,Johnny,Team Lotus,7,NaT
7,1994,1,Race,martini,Martini,Pierluigi,Minardi,8,NaT
8,1994,1,Race,comas,Comas,Érik,Larrousse,9,NaT
9,1994,1,Race,lamy,Lamy,Pedro,Team Lotus,10,NaT


### Develop Approach for Events Object

In [32]:
df_raw = pd.DataFrame(
    session_quali.event[["RoundNumber", "Location", "EventDate", "Country"]]
)

df_processed = df_raw.T.reset_index(drop=True)
df_processed.columns = df_processed.columns.str.lower()
df_processed.rename(
    columns={
        "roundnumber": "round",
        "location": "circuit_name",
        "country": "circuit_country",
    },
    inplace=True,
)
df_processed["year"] = df_processed["eventdate"].dt.year
df_processed["round"] = df_processed["round"].astype(int)

df_event_final = df_processed[
    [
        "year",
        "round",
        "circuit_name",
        "circuit_country"
    ]
]

df_event_final

,year,round,circuit_name,circuit_country
0,1994,1,São Paulo,Brazil


### Develop Approach for De-Normalized Table

In [33]:
df_event_denorm = df_event_final.merge(pd.concat([df_quali_final, df_race_final], ignore_index=True), on=["year", "round"], how="outer")

year                           int64
round                          int64
circuit_name                  object
circuit_country               object
id_driver                     object
name_driver_last              object
name_driver_first             object
name_team                     object
session                       object
position                      object
time                 timedelta64[ns]
dtype: object
   year  round circuit_name circuit_country           id_driver  \
0  1994      1    São Paulo          Brazil               senna   
1  1994      1    São Paulo          Brazil  michael_schumacher   
2  1994      1    São Paulo          Brazil               alesi   
3  1994      1    São Paulo          Brazil          damon_hill   
4  1994      1    São Paulo          Brazil            frentzen   

  name_driver_last name_driver_first name_team session position  \
0            Senna            Ayrton  Williams      Q1      1.0   
1       Schumacher           Michael  Bene

### Develop Approach for Normalized Tables

In [34]:
df_sessions = pd.concat([df_quali_final, df_race_final], ignore_index=True)
print(df_sessions.head())
print(df_sessions.tail())

   year  round           id_driver name_driver_last name_driver_first  \
0  1994      1               senna            Senna            Ayrton   
1  1994      1  michael_schumacher       Schumacher           Michael   
2  1994      1               alesi            Alesi              Jean   
3  1994      1          damon_hill             Hill             Damon   
4  1994      1            frentzen         Frentzen      Heinz-Harald   

  name_team session position                   time  
0  Williams      Q1      1.0 0 days 00:01:15.962000  
1  Benetton      Q1      2.0 0 days 00:01:16.290000  
2   Ferrari      Q1      3.0 0 days 00:01:17.385000  
3  Williams      Q1      4.0 0 days 00:01:17.554000  
4    Sauber      Q1      5.0 0 days 00:01:17.806000  
     year  round     id_driver name_driver_last name_driver_first  name_team  \
101  1994      1        berger           Berger           Gerhard    Ferrari   
102  1994      1       beretta          Beretta           Olivier  Larrousse 

#### events table

In [35]:
df_events = df_event_final[["year", "round", "circuit_name"]].reset_index(drop=True)

year             int64
round            int64
circuit_name    object
dtype: object

#### drivers table

In [36]:
df_drivers = df_sessions[["id_driver", "name_driver_last", "name_driver_first"]].drop_duplicates().reset_index(drop=True)

id_driver            object
name_driver_last     object
name_driver_first    object
dtype: object

#### teams table

In [37]:
df_teams = df_sessions[["name_team", "year", "id_driver"]].drop_duplicates().reset_index(drop=True)

name_team    object
year          int64
id_driver    object
dtype: object

#### circuits table

In [38]:
df_circuits = df_event_final[["circuit_name", "circuit_country"]].reset_index(drop=True)

circuit_name       object
circuit_country    object
dtype: object

#### results table

In [39]:
df_results = df_sessions[["year", "round", "id_driver", "name_team", "session", "position", "time"]].reset_index(drop=True)

year                   int64
round                  int64
id_driver             object
name_team             object
session               object
position              object
time         timedelta64[ns]
dtype: object